<a href="https://colab.research.google.com/github/suyashmarathe512/suyash/blob/main/CBT_based_Ai_therapist_Using_Llama3_3_70b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_groq langchain_core langchain_community pypdf  chromadb sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [4]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is lord Ram?")
print(result.content)


Lord Rama is a major deity in Hinduism and is considered one of the most important figures in Hindu mythology. He is the seventh avatar (incarnation) of the god Vishnu and is revered as a symbol of righteousness, duty, and virtue.

According to the Hindu epic, the Ramayana, Lord Rama was a king of Ayodhya, a city in ancient India. He was born to King Dasharatha and Queen Kausalya, and was the eldest of four brothers, including Lakshmana, Bharata, and Shatrughna.

The story of Lord Rama is as follows:

* Lord Rama was exiled to the forest for 14 years by his stepmother, Queen Kaikeyi, who wanted her own son, Bharata, to become the king.
* During his exile, Lord Rama's wife, Sita, was abducted by the demon king Ravana, who took her to his kingdom in Lanka (modern-day Sri Lanka).
* Lord Rama, along with his brother Lakshmana and the monkey god Hanuman, embarked on a journey to rescue Sita and defeat Ravana.
* After a long and arduous battle, Lord Rama defeated Ravana and rescued Sita, and

In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/sample_data/data", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain




In [10]:
def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()


Intializing Chatbot.........

Human: Hi, I am suyash
Chatbot: Hello Suyash! It's lovely to connect with you. I see you've been reading about the concept of mental health and its various definitions. That's a great topic to explore. The World Health Organization's definition of positive mental health is indeed a good starting point, emphasizing the importance of purpose, relationships, self-respect, and mastery.

However, as you've noted, this definition can be considered incomplete, as it doesn't take into account the social and physical environments that influence our lives. The more rounded definition you mentioned, which highlights the capacity of individuals and groups to interact with one another and the environment in ways that promote subjective wellbeing, is a more comprehensive approach.

It's also interesting to consider how people's individual interpretations of what a good life is can vary greatly. What does a good life mean to you, Suyash? What are some things that bring y